# Building a Simple PC with RPCircuits

In [1]:
using Pkg
Pkg.activate("/home/jonasrlg/code/RPCircuits.jl/")
using RPCircuits, Random
using Distributions: rand, truncated, Normal

  Activating project at `~/code/RPCircuits.jl`
┌ Info: Precompiling RPCircuits [a494de23-34c1-4aeb-b541-d9435dced8c8]
└ @ Base loading.jl:1423


The easyest way to build a PCs with RPCircuits uses a bottom-up strategy, definin first the `leaf` nodes, then `product` nodes, and lastly `sum` nodes. 

As an example, we start building a simple Circuit with 8 indicator (leafs), 4 products and 1 sum node.

![basic](pc_example_structure.png)

First we have the foundation of the PC, the **leaf** nodes. In this case the variables **a**, **b** and their respective negations **na** and **nb**. To declare an indicator leaf node using RPCircuits, we call the function `Indicator()`, that needs **two** arguments. The first argument simply refers to the variable's `scope` (the indices of the variables associated with the Indicator node). The second argument, corresponds to the `value` such that the Indicator node outputs `true` (if `value = 1.0`, the Indicator node outputs `true` only when its input is `1.0`). Using the packag we have

In [2]:
a, na, b, nb = Indicator(1, 1.0), Indicator(1, 0.), Indicator(2, 1.), Indicator(2, 0.)

(indicator 1 1.0, indicator 1 0.0, indicator 2 1.0, indicator 2 0.0)

Next, we have the second layer with 4 **product** nodes. To define a product `P`, we only need to call the function `Product(v)`, where `v`  is the vector containing all children of `P`. Hence, we can build the four products of our PC by

In [3]:
P1, P2, P3, P4 = Product([a,b]), Product([a,nb]), Product([na,b]), Product([na,nb])

(* 1 2, * 1 2, * 1 2, * 1 2)

At last, we have the **sum** node. To define a sum  node `S`, we have to call the function `Sum(v, w)`, where `v` is the vector of children of `S`; and `w` is the vector of corresponding weights. This can easily be done by

In [4]:
S = Sum([P1, P2, P3, P4], [0.4, 0.3, 0.2, 0.1])

Circuit with 9 nodes (1 sum, 4 products, 4 leaves) and 2 variables:
  1 : + 1 0.4 2 0.3 3 0.2 4 0.1
  2 : * 1 2
  3 : * 1 2
  4 : indicator 1 0.0
  5 : * 1 2
  6 : indicator 2 0.0
  7 : * 1 2
  8 : indicator 2 1.0
  9 : indicator 1 1.0

Hence, we have the circuit
![basic](spn_example.png)

To see the `scope` of a circuit rooted at a node `C`, we can type `scope(C)`. Therefore, the scope of our circuit is

In [5]:
scope(S)

BitSet with 2 elements:
  1
  2

In [6]:
x = 0
println("Manul marginalization: ", S(x,0), " + ", S(x,1))

Manul marginalization: 0.10000000000000002 + 0.2


In [7]:
println("Marginalization using inference: ", marginalize(S,[x],[1])

LoadError: syntax: incomplete: premature end of input

In [8]:
y = 1
println("Manul marginalization: ", S(0,y), " + ", S(1,y))

Manul marginalization: 0.2 + 0.4


In [9]:
println("Marginalization using inference: ", marginalize(S,[y],[2])

LoadError: syntax: incomplete: premature end of input

Using RPCircuits, it is possible to randomly sample complete configurations of the variables associated with a circuit `C`. We can do this using the function `rand(C)`, that creates a sample according to the probability defined by the PC.

In [10]:
sample = rand(S)

2-element Vector{Float64}:
 1.0
 1.0

Passing a positive integer `N` to `rand(C, N)`, creates `N` random samples.

In [11]:
Random.seed!(42)
N = 1_000
D = rand(S, N)

1000×2 Matrix{Float64}:
 1.0  0.0
 1.0  0.0
 1.0  0.0
 0.0  1.0
 1.0  0.0
 1.0  1.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  1.0
 1.0  0.0
 1.0  0.0
 1.0  1.0
 ⋮    
 1.0  1.0
 0.0  1.0
 1.0  0.0
 0.0  0.0
 1.0  1.0
 0.0  0.0
 1.0  1.0
 1.0  1.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 0.0  1.0

With the function `NLL(S, D)`, we have the `Negative Log-Likelihood` of the PC `S` w.r.t the dataset `D`.

In [12]:
println("Original model NLL = ", NLL(S, D))

Original model NLL = 1.2905776805822853


Suppose that we have an initial circuit `Sem = Sum([P1, P2, P3, P4], [0.25, 0.25, 0.25, 0.25])` and we want to learn the function `S` (such that configurations `(a,b)`, `(a,nb)`, `(na,b)` and `(na, nb)` have respective probabilities `0.4`, `0.3`, `0.2` and `0.1`). Firstly, we can check the initial `NLL` of our model `Sem` in relation to the dataset `D`.

In [13]:
Sem = Sum([P1, P2, P3, P4], [0.25, 0.25, 0.25, 0.25])
println("Initial NLL = ", NLL(Sem, D))

Initial NLL = 1.3862943611198644


Now, we can pass both our circuit `Sem` and the dataset `D` as an input to the `EM` algorithm (Expectation-Maximization algorithm, more to know about it [here][murphy]). To do this, we first define the learner `L = SEM(S)`. Then, we have `m` calls of the `update` function, for `m` iterations of the `EM` algorithm.

[murphy]: https://probml.github.io/pml-book/book1.html

In [14]:
Lem = SEM(Sem)

for i = 1:50
    update(Lem, D)
end

In [15]:
Sem.weights'

1×4 adjoint(::Vector{Float64}) with eltype Float64:
 0.381987  0.317993  0.192006  0.108014

At last, we can apply the `NLL` function another time, to see the improvement obtained by the learning process.

In [16]:
println("Final NLL = ", NLL(Sem, D))

Final NLL = 1.2891629841331282


Similarly, we can use the Gradient Descent algorithm (more to know about it [here][murphy]) to learn a circuit `Sgrad` w.r.t the dataset `D`. In this process, we have a sligthly different approach, because we initalize the sum-weights near zero (more to know about it [here][trapp]).

[murphy]: https://probml.github.io/pml-book/book1.html
[trapp]: https://arxiv.org/abs/1905.08196

In [17]:
Random.seed!(42)

# Incialize weigths close to zero
w = rand(truncated(Normal(0, 0.001), 0, Inf), 4)
print("w = $w")

Sgrad = Sum([P1, P2, P3, P4], w)

# ';' hides output
Lgrad = Gradient(Sgrad);

w = [0.0007883556016042918, 0.0006316208311167526, 0.0014386832757114134, 0.000796126919278033]

Since the circuit `Sgrad` is not normalized, we need to compute its **normalizing constant**. We do this by using the function `log_norm_const` that outputs the `log` of the normalizing constant.

In [18]:
# Normalizing Constant of the circuit0
norm_const = log_norm_const(Sgrad)

-5.6117175656758524

Now, it is possible to obtain the real `NLL` of `Sgrad` w.r.t `D` by adding `norm_const` to `NLL(Sgrad, D)`

In [19]:
println("Initial NLL = ", NLL(Sgrad, D) + norm_const)

Initial NLL = 1.48777761083484


Finally, we can apply the Gradient Descent algorithm to `Sgrad` w.r.t `D` and then see the improvement obtained by the learning process.

In [20]:
for i = 1:50
    update(Lgrad, D; learningrate=0.01)
    println("Score = ", Lgrad.score)
end

Score = 1.4877776108347467
Score = 1.3263978422853493
Score = 1.3263156856948672
Score = 1.326233690041726
Score = 1.3261518549872733
Score = 1.3260701801943475
Score = 1.3259886653272566
Score = 1.3259073100517744
Score = 1.3258261140351324
Score = 1.3257450769460193
Score = 1.3256641984545592
Score = 1.3255834782323168
Score = 1.325502915952283
Score = 1.3254225112888667
Score = 1.3253422639178938
Score = 1.3252621735165913
Score = 1.3251822397635866
Score = 1.3251024623388952
Score = 1.3250228409239158
Score = 1.3249433752014241
Score = 1.3248640648555634
Score = 1.3247849095718394
Score = 1.3247059090371067
Score = 1.3246270629395753
Score = 1.3245483709687853
Score = 1.3244698328156173
Score = 1.3243914481722743
Score = 1.3243132167322778
Score = 1.3242351381904616
Score = 1.324157212242964
Score = 1.324079438587223
Score = 1.3240018169219665
Score = 1.3239243469472102
Score = 1.3238470283642418
Score = 1.3237698608756245
Score = 1.3236928441851916
Score = 1.3236159779980192
Score

In [21]:
Sgrad.weights'

1×4 adjoint(::Vector{Float64}) with eltype Float64:
 4.69507  4.86952  1.3493  1.34121

In [22]:
normalize_circuit!(Sgrad) # Normalizing of circuit weights

In [23]:
Sgrad.weights'

1×4 adjoint(::Vector{Float64}) with eltype Float64:
 0.383112  0.397347  0.110101  0.109441

In [24]:
println("Final NLL = ", NLL(Sgrad, D))

Final NLL = 1.3225554920080314
